In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


## View Results from Hyperparameter Tuning For Current UPDRS
Using the protein and peptide data as well as the visit month, predict the UPDRS value as either Mild, Moderate, or Severe

In [8]:
# read in the data from the csv file for xgboost hyperparameter tuning
xgb_hyperparams_df = pd.read_csv('../data/processed/xgboost_extrafeats_future_cat_hyperparam_results.csv', index_col=0)
lgb_hyperparams_df = pd.read_csv('../data/processed/lgboost_extrafeats_future_cat_hyperparam_results.csv', index_col=0)

In [9]:
lgb_hyperparams_df

,updrs_1,updrs_2,updrs_3
colsample_bytree,0.582591,0.534802,0.981408
learning_rate,0.657714,0.152731,0.369053
max_depth,5.000000,7.000000,4.000000
min_child_weight,1.389338,8.496093,0.716661
min_split_gain,0.001299,0.000076,0.000254
reg_alpha,7.847890,3.242325,1.426564
reg_lambda,5.143446,1.899508,9.592843
subsample,0.754802,0.520609,0.672934


In [10]:
xgb_hyperparams_df

,updrs_1,updrs_2,updrs_3
colsample_bytree,0.788955,0.767595,0.864421
gamma,0.087901,0.001295,0.001502
learning_rate,0.530529,0.527746,0.771962
max_depth,4.000000,1.000000,7.000000
min_child_weight,0.236442,0.918564,2.586700
reg_alpha,2.299038,0.273245,4.413904
reg_lambda,1.647466,2.383878,2.901294
subsample,0.621922,0.730400,0.768483


In [11]:
# read in the protein and updrs data
updrs1_df = pd.read_csv('../data/processed/train_updrs_1_cat.csv')
updrs2_df = pd.read_csv('../data/processed/train_updrs_2_cat.csv')
updrs3_df = pd.read_csv('../data/processed/train_updrs_3_cat.csv')


train_data_new_feats = pd.read_csv("../data/processed/train_data_new_feats.csv")

updrs1_cols = [
        "visit_id",
        "patient_id",
        "visit_month",
        "updrs_1",
        "num_prot_pep",
        "num_prot",
        "num_pept",
        "updrs_1_cat",
    ]
updrs2_cols = [
        "visit_id",
        "patient_id",
        "visit_month",
        "updrs_2",
        "num_prot_pep",
        "num_prot",
        "num_pept",
        "updrs_2_cat",
    ]
updrs3_cols = [
        "visit_id",
        "patient_id",
        "visit_month",
        "updrs_3",
        "num_prot_pep",
        "num_prot",
        "num_pept",
        "updrs_3_cat",
    ]

new_updrs1_df = updrs1_df[updrs1_cols].merge(
        train_data_new_feats, how="left", on=["visit_id", "patient_id"]
    )
new_updrs2_df = updrs2_df[updrs2_cols].merge(
        train_data_new_feats, how="left", on=["visit_id", "patient_id"]
    )
new_updrs3_df = updrs3_df[updrs3_cols].merge(
        train_data_new_feats, how="left", on=["visit_id", "patient_id"]
    )



In [12]:
new_updrs1_df['updrs_1_cat'].value_counts()

mild        854
moderate    199
severe       15
Name: updrs_1_cat, dtype: int64

In [13]:
new_updrs2_df['updrs_2_cat'].value_counts()

mild        910
moderate    158
Name: updrs_2_cat, dtype: int64

In [14]:
new_updrs3_df['updrs_3_cat'].value_counts()

mild        880
moderate    168
severe       10
Name: updrs_3_cat, dtype: int64

In [15]:
# replace the categorical updrs scores with numerical for mild, moderate and severe
## combine the moderate and severe categories since there are very few severe observations
new_updrs1_df['updrs_1_cat'] = new_updrs1_df['updrs_1_cat'].map({'mild': 0, 'moderate': 1, 'severe': 1})
new_updrs2_df['updrs_2_cat'] = new_updrs2_df['updrs_2_cat'].map({'mild': 0, 'moderate': 1, 'severe': 1})
new_updrs3_df['updrs_3_cat'] = new_updrs3_df['updrs_3_cat'].map({'mild': 0, 'moderate': 1, 'severe': 1})

In [16]:
new_updrs3_df['updrs_3_cat'].value_counts()

0    880
1    178
Name: updrs_3_cat, dtype: int64

In [17]:
new_updrs3_df.columns

Index(['visit_id', 'patient_id', 'visit_month', 'updrs_3', 'num_prot_pep',
       'num_prot', 'num_pept', 'updrs_3_cat', 'kfold', 'O00391',
       ...
       'Q9UHG2_Q9UKV8', 'Q9UHG2_Q9UNU6', 'Q9UHG2_Q9Y646', 'Q9UHG2_Q9Y6R7',
       'Q9UKV8_Q9UNU6', 'Q9UKV8_Q9Y646', 'Q9UKV8_Q9Y6R7', 'Q9UNU6_Q9Y646',
       'Q9UNU6_Q9Y6R7', 'Q9Y646_Q9Y6R7'],
      dtype='object', length=25887)

In [18]:

def cross_fold_validation(df, model, target):

    updrs_results = dict()
    
    for fold in range(0, 5):
        # get the train and test data for the current fold
        train = df[df['kfold'] != fold].reset_index(drop=True)
        test = df[df['kfold'] == fold].reset_index(drop=True)

        # get the train and test data for the current fold
        drop_cols = ['visit_id', 'patient_id', f'{target}', 'kfold', f'{target}_cat']
        X_train = train.drop(columns=drop_cols)
        y_train = train[f'{target}_cat']
        X_test = test.drop(columns=drop_cols)
        y_test = test[f'{target}_cat']

        # train the model
        model.fit(X_train, y_train)

        # make predictions
        preds = model.predict(X_test)


        # save the results
        updrs_results[f'{target}_fold_{fold}'] = {
            'auc_score': roc_auc_score(y_test, preds),
            'acc_score': accuracy_score(y_test, preds),
            'precision_score': precision_score(y_test, preds),
            'recall_score': recall_score(y_test, preds),
        }
        
    mean_auc = np.mean([updrs_results[f'{target}_fold_{fold}']['auc_score'] for fold in range(0, 5)])
    mean_acc = np.mean([updrs_results[f'{target}_fold_{fold}']['acc_score'] for fold in range(0, 5)])
    mean_precision = np.mean([updrs_results[f'{target}_fold_{fold}']['precision_score'] for fold in range(0, 5)])
    mean_recall = np.mean([updrs_results[f'{target}_fold_{fold}']['recall_score'] for fold in range(0, 5)])
    
    return mean_auc, mean_acc, mean_precision, mean_recall
        
    
    

In [19]:
def prepare_xgboost_model(xgb_hyperparams_df, target):
    # train the model using the hyperparameters from the hyperparameter tuning
    updrs_hp = xgb_hyperparams_df[target].to_dict()
    updrs_hp['max_depth'] = int(updrs_hp['max_depth'])
    model = XGBClassifier(**updrs_hp)
    return model

In [20]:
# test the model function
# model = prepare_xgboost_model(xgb_hyperparams_df, 'updrs_1')
# model.get_params()

In [21]:
xgb_results = dict()

for updrs, df in zip(['updrs_1', 'updrs_2', 'updrs_3'], [new_updrs1_df, new_updrs2_df, new_updrs3_df]):
    model = prepare_xgboost_model(xgb_hyperparams_df, updrs)
    auc, acc, prec, recall = cross_fold_validation(df, model, updrs)
    xgb_results[updrs] = {"auc":auc,
                        "acc":acc,
                        "prec":prec,
                        "recall":recall}

c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:12:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:13:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[05:19:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:19:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:20:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [22]:
xgb_results

{'updrs_1': {'auc': 0.5762656608043825,
  'acc': 0.8051950331271115,
  'prec': 0.520294968189705,
  'recall': 0.19348410044062217},
 'updrs_2': {'auc': 0.5614549544471271,
  'acc': 0.8417884252555833,
  'prec': 0.4120663650075415,
  'recall': 0.16236754802931272},
 'updrs_3': {'auc': 0.5487669466706919,
  'acc': 0.8260557084846623,
  'prec': 0.4059090909090909,
  'recall': 0.13054615871953332}}

## LightGBM Classifier Results

In [23]:
def prepare_lgboost_model(lgb_hyperparams_df, target):
    # train the model using the hyperparameters from the hyperparameter tuning
    updrs_hp = lgb_hyperparams_df[target].to_dict()
    updrs_hp['max_depth'] = int(updrs_hp['max_depth'])
    model = LGBMClassifier(**updrs_hp)
    return model

In [24]:
lgb_hyperparams_df.head()

,updrs_1,updrs_2,updrs_3
colsample_bytree,0.582591,0.534802,0.981408
learning_rate,0.657714,0.152731,0.369053
max_depth,5.000000,7.000000,4.000000
min_child_weight,1.389338,8.496093,0.716661
min_split_gain,0.001299,0.000076,0.000254


In [25]:
lgb_results = dict()

for updrs, df in zip(['updrs_1', 'updrs_2', 'updrs_3'], [new_updrs1_df, new_updrs2_df, new_updrs3_df]):
    model = prepare_lgboost_model(lgb_hyperparams_df, updrs)
    auc, acc, prec, recall = cross_fold_validation(df, model, updrs)
    lgb_results[updrs] = {"auc":auc,
                        "acc":acc,
                        "prec":prec,
                        "recall":recall}

In [26]:
lgb_results

{'updrs_1': {'auc': 0.5500545941798992,
  'acc': 0.7921021455837829,
  'prec': 0.44117647058823534,
  'recall': 0.1457450761798588},
 'updrs_2': {'auc': 0.5490609023329611,
  'acc': 0.8614540827519634,
  'prec': 0.7166666666666667,
  'recall': 0.10472742127153892},
 'updrs_3': {'auc': 0.544034674980207,
  'acc': 0.8373815208657398,
  'prec': 0.6247619047619047,
  'recall': 0.1017794087763128}}

In [27]:
train_df = new_updrs1_df[new_updrs1_df['kfold'] != 4].reset_index(drop=True)
test_df = new_updrs1_df[new_updrs1_df['kfold'] == 4].reset_index(drop=True)
X_train = train_df.drop(columns=['visit_id', 'patient_id', 'updrs_1', 'kfold', 'updrs_1_cat'])
y_train = train_df['updrs_1_cat']
X_test = test_df.drop(columns=['visit_id', 'patient_id', 'updrs_1', 'kfold', 'updrs_1_cat'])
y_test = test_df['updrs_1_cat']

model = prepare_lgboost_model(lgb_hyperparams_df, 'updrs_1')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

test_df['preds'] = y_pred


In [28]:
model = prepare_lgboost_model(lgb_hyperparams_df, 'updrs_1')
model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 0.5825913965503184,
 'importance_type': 'split',
 'learning_rate': 0.6577141319875366,
 'max_depth': 5,
 'min_child_samples': 20,
 'min_child_weight': 1.3893377197890882,
 'min_split_gain': 0.0012986100523735,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 7.84789041763128,
 'reg_lambda': 5.143445719389354,
 'silent': 'warn',
 'subsample': 0.7548022099472674,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [29]:
test_df['preds'].value_counts()

0    204
1      9
Name: preds, dtype: int64

In [30]:
wrong_preds = test_df[test_df['updrs_1_cat'] != test_df['preds']]
wrong_preds.shape

(45, 25888)

In [31]:
wrong_preds['updrs_1_cat'].value_counts()

1    39
0     6
Name: updrs_1_cat, dtype: int64

## View XGBoost Results with Forecasting the UPDRS

In [32]:
# get the max category for each patient
max_df = new_updrs1_df.groupby(['patient_id'])['updrs_1_cat'].max().reset_index()
max_df = max_df.rename(columns={'updrs_1_cat': 'updrs_1_max_cat'})
# merge the max category with the original dataframe
new_updrs1_df = new_updrs1_df.merge(max_df, on=['patient_id'], how='left')
# take only the visit months that are 12 or less
updrs1_yr_df = new_updrs1_df[new_updrs1_df['visit_month'] <= 12]
updrs1_yr_df = updrs1_yr_df.drop(columns=['updrs_1_cat'])
updrs1_yr_df.rename(columns={'updrs_1_max_cat': 'updrs_1_cat'}, inplace=True)

In [33]:
# get the max category for each patient
max_df = new_updrs2_df.groupby(['patient_id'])['updrs_2_cat'].max().reset_index()
max_df = max_df.rename(columns={'updrs_2_cat': 'updrs_2_max_cat'})
# merge the max category with the original dataframe
new_updrs2_df = new_updrs2_df.merge(max_df, on=['patient_id'], how='left')
# take only the visit months that are 12 or less
updrs2_yr_df = new_updrs2_df[new_updrs2_df['visit_month'] <= 12]
updrs2_yr_df = updrs2_yr_df.drop(columns=['updrs_2_cat'])
updrs2_yr_df.rename(columns={'updrs_2_max_cat': 'updrs_2_cat'}, inplace=True)

In [34]:
# get the max category for each patient
max_df = new_updrs3_df.groupby(['patient_id'])['updrs_3_cat'].max().reset_index()
max_df = max_df.rename(columns={'updrs_3_cat': 'updrs_3_max_cat'})
# merge the max category with the original dataframe
new_updrs3_df = new_updrs3_df.merge(max_df, on=['patient_id'], how='left')
# take only the visit months that are 12 or less
updrs3_yr_df = new_updrs3_df[new_updrs3_df['visit_month'] <= 12]
updrs3_yr_df = updrs3_yr_df.drop(columns=['updrs_3_cat'])
updrs3_yr_df.rename(columns={'updrs_3_max_cat': 'updrs_3_cat'}, inplace=True)

In [35]:
xgb_forecast_hyperparams_df = pd.read_csv('../data/processed/xgboost_extrafeats_future_cat_hyperparam_results.csv', index_col=0)
lgb_forecast_hyperparams_df = pd.read_csv('../data/processed/lgboost_extrafeats_future_cat_hyperparam_results.csv', index_col=0)

In [36]:
lgb_forecast_hyperparams_df

,updrs_1,updrs_2,updrs_3
colsample_bytree,0.582591,0.534802,0.981408
learning_rate,0.657714,0.152731,0.369053
max_depth,5.000000,7.000000,4.000000
min_child_weight,1.389338,8.496093,0.716661
min_split_gain,0.001299,0.000076,0.000254
reg_alpha,7.847890,3.242325,1.426564
reg_lambda,5.143446,1.899508,9.592843
subsample,0.754802,0.520609,0.672934


In [37]:
xgb_forecast_hyperparams_df

,updrs_1,updrs_2,updrs_3
colsample_bytree,0.788955,0.767595,0.864421
gamma,0.087901,0.001295,0.001502
learning_rate,0.530529,0.527746,0.771962
max_depth,4.000000,1.000000,7.000000
min_child_weight,0.236442,0.918564,2.586700
reg_alpha,2.299038,0.273245,4.413904
reg_lambda,1.647466,2.383878,2.901294
subsample,0.621922,0.730400,0.768483


In [38]:
xgb_forecast_results = dict()

for updrs, df in zip(['updrs_1', 'updrs_2', 'updrs_3'], [updrs1_yr_df, updrs2_yr_df, updrs3_yr_df]):
    model = prepare_xgboost_model(xgb_forecast_hyperparams_df, updrs)
    print(f'UPDRS: {updrs}')
    print(f'Hyperparameters: {model.get_params()}')
    print('\n')
    auc, acc, prec, recall = cross_fold_validation(df, model, updrs)
    xgb_forecast_results[updrs] = {"auc":auc,
                        "acc":acc,
                        "prec":prec,
                        "recall":recall}

UPDRS: updrs_1
Hyperparameters: {'objective': 'binary:logistic', 'use_label_encoder': True, 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.7889550744051059, 'gamma': 0.0879005784891575, 'gpu_id': None, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': 0.5305292874654182, 'max_delta_step': None, 'max_depth': 4, 'min_child_weight': 0.2364420495186685, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 2.299038235392449, 'reg_lambda': 1.6474661293009238, 'scale_pos_weight': None, 'subsample': 0.6219217800767055, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}




c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:25:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:25:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[05:28:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:29:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:29:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [39]:
xgb_forecast_results

{'updrs_1': {'auc': 0.5750637667310504,
  'acc': 0.612134241761874,
  'prec': 0.5053968776794864,
  'recall': 0.40211615039201243},
 'updrs_2': {'auc': 0.6332869106706734,
  'acc': 0.7154773971672925,
  'prec': 0.5601709401709402,
  'recall': 0.40265476525538446},
 'updrs_3': {'auc': 0.5519635177009493,
  'acc': 0.5922675208389494,
  'prec': 0.46733614884476954,
  'recall': 0.3698171869170501}}

## LGBoost Future Categorical Predictions

In [40]:
lgb_forecast_results = dict()

for updrs, df in zip(['updrs_1', 'updrs_2', 'updrs_3'], [updrs1_yr_df, updrs2_yr_df, updrs3_yr_df]):
    model = prepare_lgboost_model(lgb_forecast_hyperparams_df, updrs)
    print(f'UPDRS: {updrs}')
    print(model.get_params())
    print('\n')
    auc, acc, prec, recall = cross_fold_validation(df, model, updrs)
    lgb_forecast_results[updrs] = {"auc":auc,
                        "acc":acc,
                        "prec":prec,
                        "recall":recall}

UPDRS: updrs_1
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.5825913965503184, 'importance_type': 'split', 'learning_rate': 0.6577141319875366, 'max_depth': 5, 'min_child_samples': 20, 'min_child_weight': 1.3893377197890882, 'min_split_gain': 0.0012986100523735, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 7.84789041763128, 'reg_lambda': 5.143445719389354, 'silent': 'warn', 'subsample': 0.7548022099472674, 'subsample_for_bin': 200000, 'subsample_freq': 0}


UPDRS: updrs_2
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.5348020886234701, 'importance_type': 'split', 'learning_rate': 0.1527311989817896, 'max_depth': 7, 'min_child_samples': 20, 'min_child_weight': 8.496093123723053, 'min_split_gain': 7.563070066811084e-05, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 3.2423247022405386, 'reg_lambda': 1.8995084845739276, 's

In [41]:
lgb_forecast_results

{'updrs_1': {'auc': 0.566277587746803,
  'acc': 0.6178930602801229,
  'prec': 0.5246477732793522,
  'recall': 0.33034213379040966},
 'updrs_2': {'auc': 0.5994839866513153,
  'acc': 0.7200334935478095,
  'prec': 0.5965909090909091,
  'recall': 0.27386326779515635},
 'updrs_3': {'auc': 0.6205309138820596,
  'acc': 0.6668198644389121,
  'prec': 0.6028421052631578,
  'recall': 0.41389545247274384}}

## Compare to Logistic Regression Model

In [43]:
model = LogisticRegression()

# run cross fold validation on the updrs 1 data
updrs1_results = cross_fold_validation(new_updrs1_df, model, 'updrs_1')

model = LogisticRegression()
updrs2_results = cross_fold_validation(new_updrs2_df, model, 'updrs_2')

model = LogisticRegression()
updrs3_results = cross_fold_validation(new_updrs3_df, model, 'updrs_3')

c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics

In [44]:
updrs1_results

(0.5, 0.7996138826729849, 0.0, 0.0)

In [45]:
updrs2_results

(0.5, 0.8520731867842569, 0.0, 0.0)

In [46]:
updrs3_results

(0.5, 0.8317254466406091, 0.0, 0.0)

## Forecast with Logistic Regression

In [47]:
model = LogisticRegression()

# run cross fold validation on the updrs 1 data
updrs1_forecast_results = cross_fold_validation(updrs1_yr_df, model, 'updrs_1')

model = LogisticRegression()
updrs2_forecast_results = cross_fold_validation(updrs2_yr_df, model, 'updrs_2')

model = LogisticRegression()
updrs3_forecast_results = cross_fold_validation(updrs3_yr_df, model, 'updrs_3')

c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dagart\anaconda3\envs\easypy37\lib\site-packages\sklearn\metrics

In [48]:
updrs1_forecast_results

(0.5, 0.6050245771498612, 0.0, 0.0)

In [49]:
updrs2_forecast_results

(0.5, 0.6944309677597058, 0.0, 0.0)

In [50]:
updrs3_forecast_results

(0.5, 0.6109384002241145, 0.0, 0.0)